# Model

### package load

In [1]:
import datetime
import numpy as np
from PIL import Image

import os
import sys
import time
from tqdm import tqdm
import numpy as np
import warnings
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms.functional as TF
import torch.nn.functional as F
from torch.autograd import Variable

from model import convert_bn_to_instancenorm, convert_bn_to_evonorm, convert_bn_to_groupnorm, DeepLabHead 
from helpers import AverageMeter, ProgressMeter, iouCalc, visim, vislbl
from labels import labels

### device

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### data load

In [14]:
npzfile = np.load('data_stand_v2.npz')

train_x = npzfile['train_x']
train_jitter = npzfile['train_x_jitter'][0:280]
train_y = npzfile['train_y']
test_x = npzfile['test_x']
test_y = npzfile['test_y']

npzfile.close()

train_x = np.vstack([train_x, train_jitter])
train_y = np.vstack([train_y, train_y[0:280]])

### DeepLab v3 ResNet50 

In [18]:
model = torchvision.models.segmentation.deeplabv3_resnet50(pretrained=False).to(device)
         # dlv3 backbone resnoet 50 :: output chnnel: 2048 --> DL input channel :: 2048
model.classifier = DeepLabHead(2048, 12).to(device) 
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=2) 

In [19]:
# Initialize metrics
best_miou = 0.0
metrics = {'train_loss' : [],
           'train_acc' : [],
           'test_acc' : [],
           'test_loss' : [],
           'miou' : []}
start_epoch = 0

### Label 

In [20]:
# Create list of class names
  # classLabels: label명
  # validClasses: label에 해당하는 id값

classLabels = []
for label in labels:
    if label.name not in classLabels:
        classLabels.append(label.name)
classLabels.append('void')

validClasses = list(
    np.unique(
        [label.id for label in labels 
         if label.id >= 0
        ] + [11]
    )
)

### train

In [21]:
## parameter

batch_size = 2
num_epoch = 5

result_dir = os.getcwd() + '/result/' + 'jitter/'

In [22]:
from torch.utils.data import TensorDataset, DataLoader

   # X: 0~255, RGB값 가지는 이미지 데이터 (560, 720, 960, 3)
   # Y: 0~11 label값 가지는 픽셀당 클래스 데이터 (560, 720, 960)
X = torch.tensor(train_x, dtype=torch.float32)
Y = torch.tensor(train_y, dtype=torch.long)

data = TensorDataset(X.permute(dims=(0, 3, 1, 2)), Y)
train_data = DataLoader(data, batch_size=batch_size, shuffle=True)

criterion = nn.CrossEntropyLoss(weight = None, ignore_index=12)

In [23]:
model.train()
res = X.shape[1] * X.shape[2] # 720 * 960

for epoch in range(num_epoch):
    
    loss_running = AverageMeter('Loss', ':.4e')
    acc_running = AverageMeter('Accuracy', ':.3f')
    
    iou = iouCalc(classLabels, validClasses, voidClass = 11)
    progress = ProgressMeter(
        len(train_data),
        [loss_running, acc_running],
        prefix="Train, epoch: [{}]".format(epoch))
    
    batch_loss = 0.0
    for batch, (x, y) in enumerate(tqdm(train_data, total=len(train_data))):
        x = x.to(device)
        y = y.to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward pass
        outputs = model(x)
        outputs = outputs['out']
        preds = torch.argmax(outputs, 1)
        
        # cross-entropy loss
        loss = criterion(outputs, y)

        # backward pass
        loss.backward()
        optimizer.step()
        
        # Statistics
        bs = x.size(0)
        loss = loss.item()
        loss_running.update(loss, bs)
        corrects = torch.sum((preds == y) & (y != 12))
        
        nvoid = int((y==12).sum()) 
        acc = corrects.double()/(bs*res-nvoid)
        acc_running.update(acc, bs)
        
        # Calculate IoU scores of current batch
        iou.evaluateBatch(preds, y)
        
        progress.display(epoch)
        
        
    scheduler.step(loss_running.avg)
    miou = iou.outputScores()
    
    # save checkpoint
    now = datetime.datetime.now()
    now_time = now.strftime('%y%m%d%H')
    
    # save path
    if not os.path.isdir(result_dir):
        os.makedirs(result_dir)
    
    save_path = result_dir
    
    with open(save_path + 'train_logs.csv', 'a') as epochs:
            epochs.write('{}, {:.4f}, {:.4f}, {:.4f}\n'.format(
                    epoch+1, loss_running.avg, acc_running.avg, miou))
            
    # Save model to file
    torch.save({
        'epoch' : epoch + 1,
        'model_state_dict' : model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'best_miou': best_miou,
        'metrics': metrics,
        }, save_path + now_time + '_E' + str(epoch+1) + '_checkpoint.pth.tar')
    
    # Save best model to file
    if miou > best_miou:
        print('mIoU improved from {:.4f} to {:.4f}.'.format(best_miou, miou))
        best_miou = miou
        
    print('epoch ', epoch)
    print('loss : {:.4f}   acc : {:.4f}   miou : {:.4f}'.format(loss_running.avg, acc_running.avg, miou))

  0%|▏                                                                               | 1/420 [00:16<1:53:38, 16.27s/it]

Train, epoch: [0][  0/420]	Loss 2.5938e+00 (2.5938e+00)	Accuracy 0.065 (0.065)


  0%|▍                                                                               | 2/420 [00:32<1:54:11, 16.39s/it]

Train, epoch: [0][  0/420]	Loss 2.4551e+00 (2.5244e+00)	Accuracy 0.152 (0.108)


  1%|▌                                                                               | 3/420 [00:48<1:53:23, 16.32s/it]

...

100%|█████████████████████████████████████████████████████████████████████████████▊| 419/420 [2:02:00<00:17, 17.49s/it]

Train, epoch: [4][  4/420]	Loss 1.3338e-01 (1.3869e-01)	Accuracy 0.952 (0.950)


100%|██████████████████████████████████████████████████████████████████████████████| 420/420 [2:02:17<00:00, 17.47s/it]

Train, epoch: [4][  4/420]	Loss 1.0120e-01 (1.3860e-01)	Accuracy 0.962 (0.950)
classes           IoU
---------------------
Pole          : 0.424
SignSymbol    : 0.730
Bicyclist     : 0.834
Pedestrian    : 0.694
Building      : 0.934
Fence         : 0.838
Pavement      : 0.906
Road          : 0.980
Car           : 0.913
Sky           : 0.935
Tree          : 0.874
---------------------
Mean IoU      : 0.824
---------------------


mIoU improved from 0.8089 to 0.8238.
epoch  4
loss : 0.1386   acc : 0.9499   miou : 0.8238


### test

In [26]:
X = torch.tensor(test_x, dtype=torch.float32)
Y = torch.tensor(test_y, dtype=torch.long)

data = torch.utils.data.TensorDataset(X.permute(dims=(0, 3, 1, 2)), Y)

test_data = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True)

save_path = result_dir
result = sorted(os.listdir(save_path), reverse=True)

In [46]:
result

['0605_best_weights.pth.tar']

In [27]:
checkpoint = torch.load(save_path + result[1]) # train, val 결과보고 결정 (overfitting 시작되기 전)
model.load_state_dict(checkpoint['model_state_dict'], strict=True)

<All keys matched successfully>

In [28]:
batch_time = AverageMeter('Time', ':6.3f')
data_time = AverageMeter('Data', ':6.3f')
progress = ProgressMeter(
    len(test_data),
    [batch_time, data_time],
    prefix='Predict: ')
loss_running = AverageMeter('Loss', ':.4e')
acc_running = AverageMeter('Accuracy', ':.3f')
    
iou = iouCalc(classLabels, validClasses, voidClass = 11)

model.eval()

batch_loss = 0.0
for batch, (x, y) in enumerate(tqdm(test_data, total=len(test_data))):

    x = x.to(device)
    y = y.to(device)

    # forward
    outputs = model(x)
    outputs = outputs['out']

    preds = torch.argmax(outputs, 1)

    # cross-entropy loss
    loss = criterion(outputs, y)

    # Statistics
    bs = x.size(0)
    loss = loss.item()
    loss_running.update(loss, bs)
    corrects = torch.sum((preds == y) & (y != 12))

    nvoid = int((y==12).sum())
    acc = corrects.double()/(bs*res-nvoid)
    acc_running.update(acc, bs)

    # Calculate IoU scores of current batch
    iou.evaluateBatch(preds, y)

miou = iou.outputScores()

print('loss : {:.4f} acc : {:.4f} miou : {:.4f}'.format(loss_running.avg, acc_running.avg, miou))

100%|██████████████████████████████████████████████████████████████████████████████████| 71/71 [07:14<00:00,  6.12s/it]

classes           IoU
---------------------
Pole          : 0.400
SignSymbol    : 0.647
Bicyclist     : 0.792
Pedestrian    : 0.646
Building      : 0.913
Fence         : 0.762
Pavement      : 0.864
Road          : 0.973
Car           : 0.899
Sky           : 0.927
Tree          : 0.837
---------------------
Mean IoU      : 0.787
---------------------
loss : 0.1985 acc : 0.9343 miou : 0.7873
